In [40]:
import pandas as pd


root_dir = '/Users/macbookpro/Downloads/groove-aligned'

In [41]:
info_clean = pd.read_csv('info_clean.csv')
energy_matrix_df = info_clean[['drummer', 'session', 'id', 'style', 'midi_filename', 'audio_filename', 'bpm', 'split']]
energy_matrix_df.head()

,drummer,session,id,style,midi_filename,audio_filename,bpm,split
0,drummer1,drummer1/eval_session,drummer1/eval_session/1,funk/groove1,drummer1/eval_session/1_funk-groove1_138_beat_...,drummer1/eval_session/1_funk-groove1_138_beat_...,138,test
1,drummer1,drummer1/eval_session,drummer1/eval_session/10,soul/groove10,drummer1/eval_session/10_soul-groove10_102_bea...,drummer1/eval_session/10_soul-groove10_102_bea...,102,test
2,drummer1,drummer1/eval_session,drummer1/eval_session/2,funk/groove2,drummer1/eval_session/2_funk-groove2_105_beat_...,drummer1/eval_session/2_funk-groove2_105_beat_...,105,test
3,drummer1,drummer1/eval_session,drummer1/eval_session/3,soul/groove3,drummer1/eval_session/3_soul-groove3_86_beat_4...,drummer1/eval_session/3_soul-groove3_86_beat_4...,86,test
4,drummer1,drummer1/eval_session,drummer1/eval_session/4,soul/groove4,drummer1/eval_session/4_soul-groove4_80_beat_4...,drummer1/eval_session/4_soul-groove4_80_beat_4...,80,test


In [45]:
import numpy as np
import torch
import librosa
import os
import matplotlib.pyplot as plt

def extract_onsets_from_audio(audio_path, sr=22050, hop_length=512):
    """
    Extract onset information from audio file
    
    Parameters:
    audio_path (str): Path to audio file
    sr (int): Sample rate
    hop_length (int): Hop length for onset detection
    
    Returns:
    onset_times (np.ndarray): Onset times in seconds
    onset_strengths (np.ndarray): Strength of each onset
    onset_backtrack (np.ndarray): Precise onset timing by backtracking
    y (np.ndarray): Audio signal
    sr (int): Sample rate
    """
    # Load audio
    y, sr = librosa.load(audio_path, sr=sr)
    
    # Compute onset strength
    onset_env = librosa.onset.onset_strength(y=y, sr=sr, hop_length=hop_length)
    
    # Detect onsets
    onset_frames = librosa.onset.onset_detect(onset_envelope=onset_env, 
                                              sr=sr, 
                                              hop_length=hop_length,
                                              backtrack=False)
    
    # Get onset times
    onset_times = librosa.frames_to_time(onset_frames, sr=sr, hop_length=hop_length)
    
    # Get precise onset times through backtracking
    onset_backtrack = librosa.onset.onset_detect(onset_envelope=onset_env,
                                               sr=sr,
                                               hop_length=hop_length,
                                               backtrack=True)
    
    onset_backtrack_times = librosa.frames_to_time(onset_backtrack, sr=sr, hop_length=hop_length)
    
    # Get onset strengths
    onset_strengths = onset_env[onset_frames]
    
    return onset_times, onset_strengths, onset_backtrack_times, y, sr

def classify_onsets_by_frequency(y, sr, onset_frames, hop_length=512, 
                               low_thresh=250, mid_thresh=2500):
    """
    Classify onsets into low, mid, and high frequency bands
    
    Parameters:
    y (np.ndarray): Audio signal
    sr (int): Sample rate
    onset_frames (np.ndarray): Frame indices of onsets
    hop_length (int): Hop length used for onset detection
    low_thresh (int): Upper threshold for low frequency band (Hz)
    mid_thresh (int): Upper threshold for mid frequency band (Hz)
    
    Returns:
    classifications (np.ndarray): Array of classifications (0: low, 1: mid, 2: high)
    energies (np.ndarray): Energy of each onset in all bands
    """
    classifications = []
    energies = []
    
    # Compute spectrogram
    D = np.abs(librosa.stft(y, hop_length=hop_length))
    
    # Map frequency bins to Hz
    freqs = librosa.fft_frequencies(sr=sr)
    
    # Get frequency band indices
    low_band = freqs <= low_thresh
    mid_band = (freqs > low_thresh) & (freqs <= mid_thresh)
    high_band = freqs > mid_thresh
    
    for frame in onset_frames:
        # Get spectrum at this frame
        spec = D[:, frame]
        
        # Calculate energy in each band
        low_energy = np.sum(spec[low_band])
        mid_energy = np.sum(spec[mid_band])
        high_energy = np.sum(spec[high_band])
        
        # Store energy in all three bands
        energies_array = np.array([low_energy, mid_energy, high_energy])
        
        # Determine dominant band
        dominant_band = np.argmax(energies_array)
        
        classifications.append(dominant_band)
        energies.append(energies_array)  # Store all three energy values
    
    return np.array(classifications), np.array(energies)

def detect_tempo_and_beats(y, sr, start_bpm=120.0):
    """
    Detect tempo and beat positions from audio
    
    Parameters:
    y (np.ndarray): Audio signal
    sr (int): Sample rate
    start_bpm (float): Initial tempo estimate
    
    Returns:
    tempo (float): Estimated tempo in BPM
    beat_frames (np.ndarray): Frame indices of beats
    beat_times (np.ndarray): Times of beats in seconds
    """
    # Detect tempo
    tempo, beat_frames = librosa.beat.beat_track(y=y, sr=sr, start_bpm=start_bpm)
    
    # Convert beat frames to times
    beat_times = librosa.frames_to_time(beat_frames, sr=sr)
    
    return tempo, beat_frames, beat_times

def quantize_onsets_to_16th_notes(onset_times, tempo, bars=1):
    """
    Quantize onsets to 16th note grid
    
    Parameters:
    onset_times (np.ndarray): Times of onsets in seconds
    tempo (float): Tempo in BPM
    bars (int): Number of bars to quantize
    
    Returns:
    quantized_steps (np.ndarray): Indices of 16th notes (0-15) for each onset
    """
    # Make sure tempo is a scalar
    if isinstance(tempo, np.ndarray):
        tempo = float(tempo)
    
    # Duration of a 16th note in seconds
    sixteenth_duration = 60.0 / (tempo * 4)
    
    # Duration of a bar in seconds
    bar_duration = 16 * sixteenth_duration
    
    # Initialize array for quantized steps
    quantized_steps = []
    
    for onset_time in onset_times:
        # Determine which bar this onset belongs to
        bar_number = int(onset_time / bar_duration)
        
        # If we've exceeded our desired number of bars, stop
        if bar_number >= bars:
            break
            
        # Time since the beginning of this bar
        time_in_bar = onset_time - (bar_number * bar_duration)
        
        # Calculate nearest 16th note (0-15) within the bar
        step_in_bar = int(np.round(time_in_bar / sixteenth_duration))
        
        # Handle rounding to exactly 16 (which should be 0 of next bar)
        if step_in_bar == 16:
            step_in_bar = 0
        
        # Only add steps that fit within our bars
        if step_in_bar < 16:
            quantized_steps.append(step_in_bar + (bar_number * 16))
    
    return np.array(quantized_steps) % 16

def audio_to_tensor_matrix(audio_path, bars_length=1, start_bpm=None, quantize=True, all_bars=False):
    """
    Create a 3-band matrix representation from audio
    
    Parameters:
    audio_path (str): Path to audio file
    bars_length (int): Number of bars to analyze
    start_bpm (float): Initial tempo estimate. If None, will be auto-detected
    quantize (bool): Whether to quantize onsets to 16th notes grid
    all_bars (bool): If True, analyze all bars in the audio file
    
    Returns:
    list: List of 3-band tensor matrices representing the drum pattern, one matrix per bar.
          Each matrix is a 3x16 tensor where the rows represent low, mid, and high frequency bands.
    """
    try:
        # Extract onset information
        onset_times, onset_strengths, precise_onset_times, y, sr = extract_onsets_from_audio(audio_path)
        
        if len(onset_times) == 0:
            raise ValueError("No onsets detected in audio")
        
        # Convert onset times to frames for frequency analysis
        onset_frames = librosa.time_to_frames(onset_times, sr=sr)
        
        # Classify onsets by frequency band
        band_classifications, band_energies = classify_onsets_by_frequency(y, sr, onset_frames)
        
        # Detect tempo and beats
        if start_bpm is None:
            tempo, beat_frames, beat_times = detect_tempo_and_beats(y, sr)
        else:
            tempo, beat_frames, beat_times = detect_tempo_and_beats(y, sr, start_bpm=start_bpm)
        
        print(f"Detected tempo: {tempo} BPM")
        
        # Make sure tempo is a scalar
        if isinstance(tempo, np.ndarray):
            tempo_value = float(tempo)
        else:
            tempo_value = tempo
            
        # Calculate total bars in the audio
        audio_duration = librosa.get_duration(y=y, sr=sr)
        bar_duration = 60.0 / tempo_value * 4  # Duration of a bar in seconds
        total_bars = int(audio_duration / bar_duration) + 1  # +1 to include partial bar
        
        # Determine how many bars to process
        if all_bars:
            bars_to_process = total_bars
            print(f"Processing all {total_bars} bars in audio")
        else:
            bars_to_process = bars_length
            print(f"Processing {bars_to_process} bars")
        
        # Create a mapping from onsets to steps
        if quantize:
            steps = quantize_onsets_to_16th_notes(onset_times, tempo_value, bars_to_process)
        else:
            # If not quantizing, estimate the step based on onset time and tempo
            sixteenth_duration = 60.0 / (tempo_value * 4)
            steps = (onset_times / sixteenth_duration).astype(int) % 16
        
        # Initialize matrices (one per bar)
        matrices = []
        for bar in range(bars_to_process):
            # Create empty 3x16 matrix for this bar
            matrix = np.zeros((3, 16))
            
            # Fill matrix based on onsets in this bar
            for i, (onset_time, classification, energy_array) in enumerate(zip(onset_times, band_classifications, band_energies)):
                # Check if this onset belongs to the current bar
                if isinstance(tempo, np.ndarray):
                    tempo_value = float(tempo)
                else:
                    tempo_value = tempo
                bar_duration = 60.0 / tempo_value * 4  # Duration of a bar in seconds
                
                if bar * bar_duration <= onset_time < (bar + 1) * bar_duration:
                    # Get the step (0-15) for this onset within the bar
                    if quantize:
                        if i < len(steps):  # Check if index is valid
                            step = steps[i]
                        else:
                            continue
                    else:
                        sixteenth_duration = 60.0 / (tempo_value * 4)
                        step = int((onset_time - bar * bar_duration) / sixteenth_duration) % 16
                    
                    # Only proceed if step is valid
                    if 0 <= step < 16:
                        # Set a threshold for detecting significant energy in each band
                        # This helps populate the mid band even if it's not dominant
                        energy_threshold = 0.3 * np.max(energy_array)
                        
                        # Add energy to all bands that have significant energy
                        for band in range(3):
                            if energy_array[band] > energy_threshold:
                                matrix[band, step] += energy_array[band]
            
            # Normalize each row to have max value of 1
            for j in range(3):
                max_val = np.max(matrix[j])
                if max_val > 0:  # Avoid division by zero
                    matrix[j] = matrix[j] / max_val
            
            # Convert numpy matrix to PyTorch tensor
            tensor_matrix = torch.tensor(matrix, dtype=torch.float32)
            matrices.append(tensor_matrix)
        
        return matrices
    
    except Exception as e:
        print(f"Error processing audio file: {e}")
        raise

def plot_3band_matrix(matrix, title="3-Band Drum Pattern"):
    """
    Visualize a 3-band matrix as a heatmap
    
    Parameters:
    matrix: numpy array or torch tensor of shape (3, 16)
    title (str): Plot title
    """
    if isinstance(matrix, torch.Tensor):
        matrix = matrix.numpy()
        
    plt.figure(figsize=(12, 4))
    plt.imshow(matrix, aspect='auto', cmap='Greys', interpolation='nearest', vmin=0, vmax=1)
    
    # Add grid to show 4 beats
    for beat in range(5):  # 0, 4, 8, 12, 16
        plt.axvline(x=beat*4-0.5, color='white', linestyle='-', linewidth=1, alpha=0.3)
    
    # Set labels
    plt.yticks([0, 1, 2], ['Low', 'Mid', 'High'])
    plt.xticks(range(16), [(i%4+1) if i%4==0 else "" for i in range(16)])
    plt.xlabel('Beat')
    plt.ylabel('Frequency Band')
    plt.title(title)
    plt.colorbar(label='Normalized Energy')
    plt.tight_layout()
    plt.show()



In [47]:
import pandas as pd
import numpy as np

def create_audio_tensor_dataframe(info_df, audio_matrices, file_index):
    """
    Creates a DataFrame from audio tensor matrices with metadata from info_df.
    
    Args:
        info_df (pd.DataFrame): DataFrame with file metadata
        audio_matrices (list): List of 3x16 numpy arrays, one for each bar
        file_index (int): Index in the info_df to get metadata
    
    Returns:
        pd.DataFrame: DataFrame with metadata and audio features
    """
    # Get metadata for the current file
    file_metadata = info_df.iloc[file_index]
    
    # Create list to hold all rows
    rows = []
    
    # Process each bar (matrix)
    for bar_num, matrix in enumerate(audio_matrices):
        # Create a row dictionary with metadata
        row = {
            'sequence': bar_num,
            'drummer': file_metadata['drummer'],
            'session': file_metadata['session'],
            'id': file_metadata['id'],
            'style': file_metadata['style'],
            'midi_filename': file_metadata['midi_filename'],
            'audio_filename': file_metadata['audio_filename'],
            'bpm': file_metadata['bpm'],
            'split': file_metadata['split']
        }
        
        # Add matrix values (low, mid, high for each step)
        for step in range(16):
            row[f'low_{step}'] = matrix[0, step]
            row[f'mid_{step}'] = matrix[1, step]
            row[f'high_{step}'] = matrix[2, step]
        
        # Add to rows list
        rows.append(row)
    
    # Create DataFrame from rows
    return pd.DataFrame(rows)

# Example usage:
# audio_df = create_audio_tensor_dataframe(energy_matrix_df, matrices, 0)

In [48]:
import pandas as pd

# Your existing code
root_dir = '/Users/macbookpro/Downloads/groove-aligned'
info_clean = pd.read_csv('info_clean.csv')
energy_matrix_df = info_clean[['drummer', 'session', 'id', 'style', 'midi_filename', 'audio_filename', 'bpm', 'split']]

# Process all files and create a combined dataframe
all_audio_df = pd.DataFrame()

for i in range(len(energy_matrix_df)):
    midi_file = f"{root_dir}/{energy_matrix_df.iloc[i]['midi_filename']}"
    wav_file = f"{root_dir}/{energy_matrix_df.iloc[i]['audio_filename']}"
    bpm = energy_matrix_df.iloc[i]['bpm']
    
    # Get matrices for this audio file
    matrices = audio_to_tensor_matrix(wav_file, start_bpm=bpm, all_bars=True)
    
    # Create dataframe for this file
    file_df = create_audio_tensor_dataframe(energy_matrix_df, matrices, i)
    
    # Append to the combined dataframe
    all_audio_df = pd.concat([all_audio_df, file_df], ignore_index=True)
    
    # Optional: print progress
    if (i + 1) % 10 == 0:
        print(f"Processed {i+1}/{len(energy_matrix_df)} files")

# Save the result
all_audio_df.to_csv('audio_tensor_database.csv', index=False)
print(f"Created database with {len(all_audio_df)} rows")

Detected tempo: [135.99917763] BPM
Processing all 16 bars in audio


/var/folders/t5/lc1wd78973j3wf4cp1zdm7gr0000gn/T/ipykernel_27321/3452499005.py:211: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  tempo_value = float(tempo)
/var/folders/t5/lc1wd78973j3wf4cp1zdm7gr0000gn/T/ipykernel_27321/3452499005.py:246: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  tempo_value = float(tempo)


Detected tempo: [99.38401442] BPM
Processing all 16 bars in audio
Detected tempo: [107.66601562] BPM
Processing all 17 bars in audio
Detected tempo: [86.1328125] BPM
Processing all 16 bars in audio
Detected tempo: [80.74951172] BPM
Processing all 17 bars in audio
Detected tempo: [112.34714674] BPM
Processing all 22 bars in audio
Detected tempo: [86.1328125] BPM
Processing all 16 bars in audio
Detected tempo: [135.99917763] BPM
Processing all 16 bars in audio
Detected tempo: [64.59960938] BPM
Processing all 16 bars in audio
Detected tempo: [103.359375] BPM
Processing all 16 bars in audio
Processed 10/1090 files
Detected tempo: [107.66601562] BPM
Processing all 39 bars in audio
Detected tempo: [117.45383523] BPM
Processing all 1 bars in audio
Detected tempo: [95.703125] BPM
Processing all 54 bars in audio
Detected tempo: [60.09265988] BPM
Processing all 52 bars in audio
Detected tempo: [95.703125] BPM
Processing all 90 bars in audio
Detected tempo: [129.19921875] BPM
Processing all 2 bar

In [31]:
fwod_y = pd.read_excel('balanced_all_data_1_bar.xlsx')
fwod_y

,file,sequence,class,step_0,step_1,step_2,step_3,step_4,step_5,step_6,step_7,step_8,step_9,step_10,step_11,step_12,step_13,step_14,step_15
0,magenta midi/pop/105_pop_132_beat_4-4.mid,0,pop,0.000000,0.000000,0.000000,0.053571,0.000000,0.000000,0.000000,1.000000,0.178571,0.071429,0.000000,0.428571,0.000000,0.058036,0.000000,0.000000
1,magenta midi/pop/105_pop_132_beat_4-4.mid,1,pop,0.897638,0.188976,0.000000,0.000000,1.000000,0.133858,0.000000,1.000000,0.000000,0.212598,0.000000,1.000000,0.000000,0.259843,0.000000,1.000000
2,magenta midi/pop/105_pop_132_beat_4-4.mid,2,pop,0.000000,0.377953,0.000000,1.000000,0.000000,0.000000,0.000000,0.157480,0.892388,0.049869,0.000000,0.889764,0.000000,0.068241,0.000000,0.774278
3,magenta midi/pop/105_pop_132_beat_4-4.mid,3,pop,0.000000,0.480315,0.000000,1.000000,0.000000,0.393701,0.000000,0.333333,0.000000,0.482940,0.000000,1.000000,0.000000,0.041995,0.000000,0.703412
4,magenta midi/pop/105_pop_132_beat_4-4.mid,4,pop,0.094488,0.433071,0.000000,1.000000,0.000000,0.000000,0.377953,0.299213,0.000000,0.419948,0.000000,0.666667,0.314961,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1021,magenta midi/jazz/47_jazz_102_beat_4-4.mid,9,jazz,0.423529,0.000000,0.000000,0.000000,0.658824,0.000000,0.768627,0.000000,0.290196,0.062745,0.298039,0.560784,0.000000,0.000000,1.000000,0.556863
1022,magenta midi/jazz/47_jazz_102_beat_4-4.mid,10,jazz,0.139303,0.258706,0.000000,0.825871,0.000000,0.000000,0.840796,0.373134,0.000000,0.000000,0.641791,0.363184,0.482587,0.149254,0.701493,1.000000
1023,magenta midi/jazz/47_jazz_102_beat_4-4.mid,11,jazz,0.000000,0.000000,0.000000,0.891089,0.861386,0.000000,0.980198,0.613861,0.000000,0.000000,0.000000,0.891089,1.000000,0.000000,0.633663,0.841584
1024,magenta midi/jazz/47_jazz_102_beat_4-4.mid,12,jazz,0.000000,0.000000,0.000000,0.954023,0.000000,0.000000,0.390805,0.390805,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.356322,0.000000
